In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import round,col, when, udf, format_number, abs, avg
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.types import DoubleType
import pp_events as pp

In [ ]:
spark = SparkSession.builder.appName("xG5").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/24 11:17:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
events = spark.read.csv('Data/events.csv', header=True, inferSchema=True,sep=';')

In [5]:
events_shot = pp.preprocessing(events,spark)

Data loaded


Spatial data calculated
Preferred foot calculated
Goal column created


Number of players inside the area calculated
Dummies created
Boolean data converted to integer


In [6]:
events_shot.columns

['id',
 'player_id',
 'shot_location_x',
 'shot_location_y',
 'distance_to_goal',
 'shot_angle',
 'preferred_foot_shot',
 'other_pp',
 'from_fk',
 'from_ti',
 'from_corner',
 'from_counter',
 'from_gk',
 'from_keeper',
 'from_ko',
 'header',
 'corner_type',
 'fk_type',
 'pk_type',
 'half_volley_technique',
 'volley_technique',
 'lob_technique',
 'overhead_technique',
 'backheel_technique',
 'diving_h_technique',
 'under_pressure',
 'shot_aerial_won',
 'shot_first_time',
 'shot_one_on_one',
 'shot_open_goal',
 'shot_follows_dribble',
 'players_inside_area',
 'shot_statsbomb_xg',
 'shot_outcome',
 'goal']

In [7]:
events_shot.show()

24/12/24 11:18:07 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/12/24 11:18:18 WARN TaskSetManager: Stage 5 contains a task of very large size (1681 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+---------+---------------+---------------+----------------+----------+-------------------+--------+-------+-------+-----------+------------+-------+-----------+-------+------+-----------+-------+-------+---------------------+----------------+-------------+------------------+------------------+------------------+--------------+---------------+---------------+---------------+--------------+--------------------+-------------------+-----------------+------------+----+
|                  id|player_id|shot_location_x|shot_location_y|distance_to_goal|shot_angle|preferred_foot_shot|other_pp|from_fk|from_ti|from_corner|from_counter|from_gk|from_keeper|from_ko|header|corner_type|fk_type|pk_type|half_volley_technique|volley_technique|lob_technique|overhead_technique|backheel_technique|diving_h_technique|under_pressure|shot_aerial_won|shot_first_time|shot_one_on_one|shot_open_goal|shot_follows_dribble|players_inside_area|shot_statsbomb_xg|shot_outcome|goal|
+-----------------

In [8]:
# since the dataframe is final, we save it into a variable in memory
events_shot = events_shot.cache()

In [9]:
events_shot.printSchema()

root
 |-- id: string (nullable = true)
 |-- player_id: double (nullable = false)
 |-- shot_location_x: float (nullable = false)
 |-- shot_location_y: float (nullable = false)
 |-- distance_to_goal: double (nullable = false)
 |-- shot_angle: float (nullable = false)
 |-- preferred_foot_shot: integer (nullable = true)
 |-- other_pp: integer (nullable = false)
 |-- from_fk: integer (nullable = false)
 |-- from_ti: integer (nullable = false)
 |-- from_corner: integer (nullable = false)
 |-- from_counter: integer (nullable = false)
 |-- from_gk: integer (nullable = false)
 |-- from_keeper: integer (nullable = false)
 |-- from_ko: integer (nullable = false)
 |-- header: integer (nullable = false)
 |-- corner_type: integer (nullable = false)
 |-- fk_type: integer (nullable = false)
 |-- pk_type: integer (nullable = false)
 |-- half_volley_technique: integer (nullable = false)
 |-- volley_technique: integer (nullable = false)
 |-- lob_technique: integer (nullable = false)
 |-- overhead_techniq

In [10]:
features = ['other_pp','from_fk','from_ti','from_corner','from_counter','from_gk','from_keeper','from_ko',
            'header','corner_type','fk_type','pk_type',
            'half_volley_technique','volley_technique','lob_technique','overhead_technique','backheel_technique','diving_h_technique',
            'distance_to_goal', 'shot_angle', 'preferred_foot_shot', 'under_pressure',
            'shot_aerial_won','shot_first_time','shot_one_on_one','shot_open_goal','shot_follows_dribble','players_inside_area']
target = ['goal']

In [11]:
input_size = len(features)
output_size = 2

In [12]:
from pyspark.ml.feature import VectorAssembler

# Define the assembler
feature_assembler = VectorAssembler(inputCols=features, outputCol="features_vector")

# Transform the dataset
assembled_data = feature_assembler.transform(events_shot)
train_data, test_data = assembled_data.randomSplit([0.7, 0.3], seed=42)

In [25]:
from pyspark.ml.classification import LogisticRegression

# Define the logistic regression model
log_reg = LogisticRegression(featuresCol="features_vector", labelCol="goal", maxIter=10)
# Create the pipeline
pipeline = Pipeline(stages=[log_reg])
# Still rerunning all, fix that
xg_model = pipeline.fit(train_data)
# Make predictions
predictions = xg_model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="goal", rawPredictionCol="rawPrediction")
roc_auc = evaluator.evaluate(predictions)
print(f"ROC-AUC: {roc_auc}")

ROC-AUC: 0.8208104305719148


In [ ]:
from pyspark.ml.classification import RandomForestClassifier

# Define the Random Forest classifier
rf = RandomForestClassifier(featuresCol="features_vector", labelCol="goal", numTrees=100)

# Create the pipeline
pipeline = Pipeline(stages=[rf])

# Fit the model
xg_model = pipeline.fit(train_data)

# Make predictions
predictions = xg_model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="goal", rawPredictionCol="rawPrediction")
roc_auc = evaluator.evaluate(predictions)
print(f"ROC-AUC: {roc_auc}")

In [42]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

# Define the Multilayer Perceptron classifier
layers = [input_size, 64, 32, output_size]  # Example layers, adjust input_size and output_size based on your data

mlp = MultilayerPerceptronClassifier(featuresCol="features_vector", labelCol="goal", maxIter=100, layers=layers, blockSize=128, seed=1234)

# Create the pipeline
pipeline = Pipeline(stages=[mlp])

# Fit the model
xg_model = pipeline.fit(train_data)

# Make predictions
predictions = xg_model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="goal", rawPredictionCol="rawPrediction")
roc_auc = evaluator.evaluate(predictions)
print(f"ROC-AUC: {roc_auc}")

ROC-AUC: 0.815963763021576


In [ ]:
#### BEST MODEL SO FAR ####
from pyspark.ml.classification import GBTClassifier

# Define the Gradient-Boosted Tree classifier
gbt = GBTClassifier(featuresCol="features_vector", labelCol="goal", maxIter=50)

# Create the pipeline
pipeline = Pipeline(stages=[gbt])

# Fit the model
xg_model = pipeline.fit(train_data)

# Make predictions
predictions = xg_model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="goal", rawPredictionCol="rawPrediction")
roc_auc = evaluator.evaluate(predictions)
print(f"ROC-AUC: {roc_auc}")

24/12/24 11:19:46 WARN TaskSetManager: Stage 28 contains a task of very large size (1681 KiB). The maximum recommended task size is 1000 KiB.
24/12/24 11:21:29 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


ROC-AUC: 0.8231003026023795


#### For Gradient Boosted Trees

In [ ]:

# Define a function to extract the probability of the goal (class 1)
def extract_goal_probability(probability):
    return float(probability[1])

# Register the function as a UDF
extract_goal_probability_udf = udf(extract_goal_probability, DoubleType())

# Create a new column with the goal probability
predictions_with_goal_prob = predictions.withColumn("goal_probability", extract_goal_probability_udf(col("probability")))

# Format the goal_probability to remove scientific notation
predictions_with_goal_prob = predictions_with_goal_prob.withColumn("goal_probability", format_number(col("goal_probability"), 10))

# Add a new column showing the difference between shot_statsbomb_xg and goal_probability
predictions_with_goal_prob = predictions_with_goal_prob.\
    withColumn("difference", format_number(col("shot_statsbomb_xg") - col("goal_probability"),10))

predictions_with_goal_prob = predictions_with_goal_prob.\
    withColumn("difference_abs", format_number(abs(col('difference')),10))

# Show the results
predictions_with_goal_prob.select("shot_statsbomb_xg", "goal_probability", "difference",'difference_abs').show(50,False)

+-----------------+----------------+-------------+--------------+
|shot_statsbomb_xg|goal_probability|difference   |difference_abs|
+-----------------+----------------+-------------+--------------+
|0.04334119       |0.0425247594    |0.0008164306 |0.0008164306  |
|0.05638644       |0.0661404310    |-0.0097539910|0.0097539910  |
|0.044450935      |0.0460373865    |-0.0015864515|0.0015864515  |
|0.03341424       |0.0387692834    |-0.0053550434|0.0053550434  |
|0.03511139       |0.0401640978    |-0.0050527078|0.0050527078  |
|0.25300345       |0.2392366206    |0.0137668294 |0.0137668294  |
|0.18661515       |0.2475244429    |-0.0609092929|0.0609092929  |
|0.049519155      |0.0834491156    |-0.0339299606|0.0339299606  |
|0.2808055        |0.2549963792    |0.0258091208 |0.0258091208  |
|0.022752915      |0.0304182598    |-0.0076653448|0.0076653448  |
|0.14442237       |0.1895965604    |-0.0451741904|0.0451741904  |
|0.26464975       |0.1615358612    |0.1031138888 |0.1031138888  |
|0.0752324

In [ ]:
# average of the difference
avg_diff = predictions_with_goal_prob.select(avg(col("difference"))).collect()[0][0]
abs_avg_diff = predictions_with_goal_prob.select(avg(col("difference_abs"))).collect()[0][0]

print(f"Average difference: {avg_diff}")
print(f"Abs average difference: {abs_avg_diff}")

Average difference: -0.008360095045299182
Abs average difference: 0.033625974251175215


In [18]:
# round the goal_probability to 0 or 1, same for the shot_statsbomb_xg and compare with goal to check the accuracy of the model
# vs the accuracy of the shot_statsbomb_xg model

predictions_with_goal_prob = predictions_with_goal_prob.withColumn("goal_probability_rounded", round(col("goal_probability")))\
    .withColumn("shot_statsbomb_xg_rounded", round(col("shot_statsbomb_xg")))

# Check if the rounded probability matches the actual goal for your model
predictions_with_accuracy = predictions_with_goal_prob.withColumn(
    "model_correct", when(col("goal") == col("goal_probability_rounded"), 1).otherwise(0))\
    .withColumn("statsbomb_correct", when(col("goal") == col("shot_statsbomb_xg_rounded"), 1).otherwise(0))

In [19]:
# Calculate accuracy
accuracy_model = predictions_with_accuracy.selectExpr("avg(model_correct) as model_accuracy").first()["model_accuracy"]
accuracy_statsbomb = predictions_with_accuracy.selectExpr("avg(statsbomb_correct) as statsbomb_accuracy").first()["statsbomb_accuracy"]

print(f"Model Accuracy: {accuracy_model * 100:.2f}%")
print(f"StatsBomb xG Accuracy: {accuracy_statsbomb * 100:.2f}%")

Model Accuracy: 90.44%
StatsBomb xG Accuracy: 90.59%


In [ ]:
# Testing matching of models, is out model as good as SB_xg?
predictions_with_accuracy = predictions_with_goal_prob.withColumn(
    "model_to_sbxg", when(col("goal_probability_rounded") == col("shot_statsbomb_xg_rounded"), 1).otherwise(0))

xg_accuracy_model = predictions_with_accuracy.selectExpr("avg(model_to_sbxg) as model_accuracy").first()["model_accuracy"]

print(f"Models matching percentage: {xg_accuracy_model * 100:.2f}%")

Models matching percentage: 98.35%


In [ ]:
# show each feature with its coefficient
coefficients = xg_model.stages[-1].coefficients
features_coefficients = list(zip(features, coefficients))

for feature, coefficient in features_coefficients:
    print(f"{feature}: {coefficient}")